In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('file.dat', sep='\s+|\s+',engine='python')
data.to_csv('file.csv', index=None) # This will convert the dat file to csv file. The file will get stored as 21MC****.csv

In [ ]:
dataset=pd.read_csv('file.csv') # Reading the file 21MC****.csv

In [ ]:
t=dataset.iloc[:,3].values # The time period
rzxx=dataset.iloc[:,4].values # Real component of zxx
rzxx=rzxx.tolist()
izxx=dataset.iloc[:,5].values # Imaginary componenet of zxx
izxx=izxx.tolist()
rzxy=dataset.iloc[:,6].values # Real component of zxy 
rzxy=rzxy.tolist()
izxy=dataset.iloc[:,7].values # Imaginary componenet of zxy
izxy=izxy.tolist()
rzyx=dataset.iloc[:,8].values # Real component of zyx
rzyx=rzyx.tolist()
izyx=dataset.iloc[:,9].values# Imaginary componenet of zyx
izyx=izyx.tolist()
rzyy=dataset.iloc[:,10].values # Real component of zyy
rzyy=rzyy.tolist()
izyy=dataset.iloc[:,11].values # Imaginary componenet of zyy
izyy=izyy.tolist()

In [ ]:
def phi_max_min(rzxx,izxx,rzxy,izxy,rzyx,izyx,rzyy,izyy):
    x=[[rzxx,rzxy],[rzyx,rzyy]] # Matrix of the real part of the impedence tensor
    x=np.array(x)
    y=[[izxx,izxy],[izyx,izyy]] # Matrix of the imagiary part of the impedence tensor
    y=np.array(y)
    phase=np.matmul((np.linalg.inv(x)),y) # phase is the phase tensor
    detphase=np.linalg.det(phase) # determenannt of the phase tensor
    phi1=(phase[0][0]+phase[1][1])/2
    phi3=(phase[0][1]-phase[1][0])/2
    if(detphase<0):
        phi2=math.sqrt(abs(detphase))
        phimin=-(math.sqrt((phi1**2)+(phi3**2)))+math.sqrt((phi1**2)+(phi3**2)-detphase)
        phimax=math.sqrt((phi1**2)+(phi3**2))+math.sqrt((phi1**2)+(phi3**2)-detphase)
    else:
        phi2=math.sqrt(detphase)
        phimin=math.sqrt((phi1**2)+(phi3**2))-math.sqrt((((phase[0][0]-phase[1][1])**2)+((phase[0][1]+phase[1][0])**2))/4)
        phimax=math.sqrt((phi1**2)+(phi3**2))+math.sqrt((((phase[0][0]-phase[1][1])**2)+((phase[0][1]+phase[1][0])**2))/4)
    if(phase[0][0]==phase[1][1]):
        alpha=math.radians(90)
    else:
        alpha=(1/2)*math.atan((phase[0][1]+phase[1][0])/(phase[0][0]-phase[1][1]))
    
    if(phi1==0):
        beta=math.radians(90)
    else:
        beta=(1/2)*math.atan(phi3/phi1)
    # phimax, phimin and beta are the coordinate invariants
    # 2*phimax will be the lenght of the semi major axis of the phase tensor ellipse
    # 2*phimin will be the lenght of the semi minor axis of the phase tensor ellipse
    # alpha-beta will be angle made by the semi major axis of the phase tensor
    # ellipse with the x-axis
    a=[phimax,phimin,alpha,beta]
    return a

In [ ]:
def elip(a,b,theta,h,k):
    # a is the half of the length of the semi major axis
    # b is the half of the length of the semi minor axis
    # theta is the angle made by the semi major axis with the X-axis
    # h and k are the coordinate of the centre of the ellipse.
    x=[]
    for i in range(int((2*a)/0.02)+1):
        x.append(-a+(i*0.02)) # This is created to make points along X-axis to plot
        # an ellipse
    y1=[]
    y2=[]
    for i in range(len(x)):
        y1.append((b/a)*math.sqrt((a**2)-(x[i]**2))) 
        y2.append(-(b/a)*math.sqrt((a**2)-(x[i]**2)))
        # For each vlaue of X-cocrdinate there will be two values of Y-coordiante 
        # for an ellipse, one is postive and the other is negative
        # y1 takes all the positive values of the Y-coordinate 
        # y2 takes all the negative values of the Y-coordinate
        # The ellipse is centered around origin and the semi major axis of it
        # is along X-axis
    y=y1+(y2[::-1]) # y2[::-1] will reverse the order in which the eleents of
    # y2 are appened
    x=x+sorted(x,reverse=True) # sorted(x,reverse=True) will  temporarly make
    # the elements of x sorted in ascending order
    r=[[math.cos(theta),-math.sin(theta)],[math.sin(theta),math.cos(theta)]]
    # r is the rotating matrix, which will rotate a matrix by an angle theta
    r=np.array(r)
    w=np.array([x,y]) # A matrix is created using the coordiantes of all the
    # points of the ellipse
    t=np.matmul(r,w) # This will rotate the ellipse by an angle theta in
    # anti clockwise direction
    u=t[0] # The X-coordiante of the rotated ellipse
    v=t[1] # The Y-coordiante of the rotated ellipse
    u=u+np.array([h]*len(u))
    v=v+np.array([k]*len(v))
    # u=u+np.array([h]*len(u)) and v=v+np.array([k]*len(v)) will make the 
    # rotated ellipse to be centered around the point (h,k)
    # u is the array of the X-coordiantes of the rotated and displaced
    # ellipse
    # v is the array of the Y-coordiantes of the rotated and displaced
    # ellipse
    a=[u,v]
    return a

In [ ]:
x=dataset.iloc[:,1].values # X-coordinates of the MT stations
y=dataset.iloc[:,2].values # Y-coordinates of the MT stations
x=x/1000 # The X-coordinates is scalled down to km
y=y/1000 # The Y-coordinates is scalled down to km
k1=plt.figure(1,figsize=(19,19))
plt.scatter(x,y,s=2,c='black') # Plot of the positions of the MT stations
plt.xlabel('x in km',fontsize=17)
plt.ylabel('y in km',fontsize=17)
plt.xticks(fontsize=17)
plt.yticks(fontsize=17)
plt.title('Fig 1: MT station map',fontsize=19)
z=[] # For the plot of rose diagram
for i in range(9): # The range is upto 9 because the number of different time periods
    # are 9
    h=[]
    fn='k'+str(i+2)
    globals()[fn]=plt.figure(i+2) # This will name each plot
    for j in range(i,len(x),9):
        c=phi_max_min(rzxx[j],izxx[j],rzxy[j],izxy[j],rzyx[j],izyx[j],rzyy[j],izyy[j])
        # Data of the different components of different impedence tensors are fed to the 
        # phi_max_min function to get phimax, phimin, alpha and beta for each MT stations 
        # and for each time period
        m=elip(c[0],c[1],c[2]-c[3],x[j],y[j]) # This will create the array of X-coordinates and 
        # Y-coordinates of the rotated and displaced ellipse for each MT station and for each 
        # time period
        u=m[0] # Array of the X-coordiantes of the rotated and displaced ellipse
        v=m[1] # Array of the Y-coordiantes of the rotated and displaced ellipse
        plt.plot(u,v,color='black',linewidth=2)
        # Plot of the rotated and displaced ellipse
        if(x[j]==y[j]):
            h.append(c[2]-c[3]) # This will take the value of alpha-beta only for the diagonal
            # MT stations
        else:
            continue
    z.append(h)
    plt.scatter(x,y,s=2,c='black') # PLot the MT stations
    plt.xlabel('x in km',fontsize=17)
    plt.ylabel('y in km',fontsize=17)
    plt.xticks(fontsize=17)
    plt.yticks(fontsize=17)
    titl='Fig '+str(i+2)+': Phase tensor elllipse superimposed in station map for time period '+str(t[i])+' Sec'
    plt.title(titl,fontsize=19)
    globals()[fn].set_figwidth(19)
    globals()[fn].set_figheight(19)
    plt.show()

In [ ]:
for i in range(len(z)):
    for j in range(len(z[i])):
        if(z[i][j]<0):
            z[i][j]=(2*np.pi)-z[i][j] # If the strike angles are less than 0 then they are
            # substracted from 2*pi radian

for i in range(9):
    n=36
    theta=np.linspace(0,2*np.pi,n) # The polar plot is devided in 36 inervals, each equal to 
    # has width of pi/18 radian
    w=[0]*len(theta)
    for j in range(len(z[i])):
        for k in range(len(theta)-1):
            if(theta[k]<=z[i][j] and z[i][j]<theta[k+1]): # To find which interval holds how
                # many numbers of angles
                w[k]=w[k]+1
            else:
                continue
    plt.figure(figsize=(12,12))
    ax=plt.subplot(111,polar='True')
    ax.set_theta_zero_location('N') # 0 radian direction is aligned along north direcion
    k=ax.bar(theta,w,width=0.2,edgecolor='k',alpha=0.5,label='bar') # Plot of rose diagram
    ax.set_theta_direction(-1) # The angles will be ploted in clockwise direction
    tilt='Fig '+str(i+11)+': Rose diagram of strike angle for time period '+str(t[i])+' Sec'
    plt.title(tilt,fontsize=19)
    plt.show()